# Easy Dev for Post-hoc OOD Detectors

This notebook integrates some simple post-hoc OOD detection methods.

We choose ImageNet-1K as in-distribution (ID) and load a pretrained vision transformer (ViT).

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Models and Dataset

In [25]:
from openood.utils import config
from openood.datasets import get_dataloader, get_ood_dataloader
from openood.evaluators import get_evaluator
from openood.networks import get_network
from torchvision.transforms import RandAugment
from torchvision import transforms
from openood.postprocessors import get_postprocessor

In [26]:
# load config files for cifar10 baseline
# PATH="/home/gridsan/nhulkund/OpenOOD/"
# config_files = [
#     './configs/datasets/cifar10/cifar10.yml',
#     './configs/datasets/cifar10/cifar10_ood.yml',
#     './configs/networks/resnet18_32x32.yml',
#     './configs/pipelines/test/test_ood.yml',
#     './configs/preprocessors/base_preprocessor.yml',
#     './configs/postprocessors/mls.yml',
#     './configs/rand_augment/no_augmentation.yml'
# ]
# config = config.Config(*config_files)
# # modify config 
# #config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
# config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/cifar10_res18_acc94.30.ckpt'
# config.network.pretrained = True
# config.num_workers = 8
# config.save_output = False
# config.parse_refs()

# load config files for cifar100 baseline
PATH="/home/gridsan/nhulkund/OpenOOD/"
config_files = [
    './configs/datasets/cifar100/cifar100.yml',
    './configs/datasets/cifar100/cifar100_ood.yml',
    './configs/networks/resnet18_32x32.yml',
    './configs/pipelines/test/test_ood.yml',
    './configs/preprocessors/base_preprocessor.yml',
    './configs/postprocessors/mls.yml',
    './configs/rand_augment/base.yml',
]
config = config.Config(*config_files)
# modify config 
#config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/cifar100_res18_acc78.20.ckpt'
config.network.pretrained = True
config.num_workers = 8
config.save_output = False
config.parse_refs()
# load config files for cifar10 baseline
# PATH="/home/gridsan/nhulkund/OpenOOD/"
# config_files = [
#     './configs/datasets/imagenet/imagenet.yml',
#     './configs/datasets/imagenet/imagenet_ood.yml',
#     './configs/networks/resnet50.yml',
#     './configs/pipelines/test/test_ood.yml',
#     './configs/preprocessors/base_preprocessor.yml',
#     './configs/postprocessors/msp.yml',
# ]
# config = config.Config(*config_files)
# # modify config 
# #config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
# config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/imagenet_res50_acc76.10.pth'
# config.network.pretrained = True
# config.num_workers = 8
# config.save_output = False
# config.parse_refs()

In [27]:
print(config)

dataset:
    image_size: 32
    interpolation: bilinear
    name: cifar100
    normalization_type: cifar100
    num_classes: 100
    num_gpus: 1
    num_machines: 1
    num_workers: 8
    pre_size: 32
    split_names: ['train', 'val', 'test']
    test:
        batch_size: 200
        data_dir: ./data/images_classic/
        dataset_class: ImglistDataset
        imglist_pth: ./data/benchmark_imglist/cifar100/test_cifar100.txt
        shuffle: False
    train:
        batch_size: 128
        data_dir: ./data/images_classic/
        dataset_class: ImglistDataset
        imglist_pth: ./data/benchmark_imglist/cifar100/train_cifar100.txt
        shuffle: True
    val:
        batch_size: 200
        data_dir: ./data/images_classic/
        dataset_class: ImglistDataset
        imglist_pth: ./data/benchmark_imglist/cifar100/val_cifar100.txt
        shuffle: False
evaluator:
    name: ood
exp_name: cifar100_resnet18_32x32_test_ood_ood_mls_default
machine_rank: 0
mark: default
merge_option: def

In [28]:
#print(config)

In [29]:
# get dataloader
id_loader_dict = get_dataloader(config)
ood_loader_dict = get_ood_dataloader(config)
# init network
net = get_network(config.network).cuda()
# init ood evaluator
evaluator = get_evaluator(config)
postprocessor = get_postprocessor(config)
# acc_metrics = evaluator.eval_acc(net, id_loader_dict['test'],
#                                          postprocessor)
# ood_metrics = evaluator.eval_ood(net, id_loader_dict, ood_loader_dict, postprocessor)

Model Loading resnet18_32x32 Completed!


In [30]:
# print(len(id_loader_dict['test'].dataset))
# print(len(id_loader_dict['val'].dataset))
# print(len(id_loader_dict['train'].dataset))
# print(len(ood_loader_dict['nearood']['tin'].dataset))

In [31]:
# nextiter=iter(id_loader_dict['test'])
# batch=next(nextiter)
# batch['data'].shape

In [32]:
# nextiter=iter(ood_loader_dict['nearood']['tin'])
# batch=next(nextiter)
# batch['data'].shape

In [33]:
acc_metrics = evaluator.eval_acc(net, id_loader_dict['test'], postprocessor)
ood_metrics = evaluator.eval_ood(net, id_loader_dict, ood_loader_dict, postprocessor)

Performing inference on cifar100 dataset...
Processing nearood...
Performing inference on cifar10 dataset...
Computing metrics on cifar10 dataset...
FPR@95: 73.33, AUROC: 82.67 AUPR_IN: 83.44, AUPR_OUT: 80.75
CCR: 1.71, 11.04, 25.80, 54.49, ACC: 77.10
──────────────────────────────────────────────────────────────────────
Performing inference on tin dataset...
Computing metrics on tin dataset...
FPR@95: 66.14, AUROC: 86.05 AUPR_IN: 89.32, AUPR_OUT: 80.33
CCR: 0.24, 12.54, 30.69, 60.13, ACC: 77.10
──────────────────────────────────────────────────────────────────────
Computing mean metrics...
FPR@95: 69.73, AUROC: 84.36 AUPR_IN: 86.38, AUPR_OUT: 80.54
CCR: 0.98, 11.79, 28.24, 57.31, ACC: 77.10
──────────────────────────────────────────────────────────────────────
Processing farood...
Performing inference on mnist dataset...
Computing metrics on mnist dataset...
FPR@95: 74.15, AUROC: 84.54 AUPR_IN: 63.77, AUPR_OUT: 96.78
CCR: 13.60, 23.97, 38.66, 59.32, ACC: 77.10
────────────────────────

In [34]:
ood_loader_dict['nearood']['tin'].dataset

## Feature Extraction

In [35]:
from tqdm import tqdm
import numpy as np
import torch
import os.path as osp
import os
import cv2
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity
from skimage import img_as_ubyte
from PIL import Image



In [36]:

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [37]:
def save_arr_to_dir(arr, dir):
    os.makedirs(os.path.dirname(dir), exist_ok=True)
    with open(dir, 'wb+') as f:
        np.save(f, arr)

In [38]:
save_root = PATH+f'/results/{config.exp_name}'


In [39]:
# save id (test & val) results
net.eval()
modes = ['test', 'val']
for mode in modes:
    dl = id_loader_dict[mode]
    dataiter = iter(dl)
    
    logits_list = []
    feature_list = []
    label_list = []
    id_list = []
    count=0
    print(id_list)
    for i in tqdm(range(1,
                    len(dataiter) + 1),
                    desc='Extracting reults...',
                    position=0,
                    leave=True):
        batch = next(dataiter)
        data = batch['data'].cuda()
        label = batch['label']
        with torch.no_grad():
            logits_cls, feature = net(data, return_feature=True)
        logits_list.append(logits_cls.data.to('cpu').numpy())
        feature_list.append(feature.data.to('cpu').numpy())
        label_list.append(label.numpy())

    logits_arr = np.concatenate(logits_list)
    feature_arr = np.concatenate(feature_list)
    label_arr = np.concatenate(label_list)
    
    save_arr_to_dir(logits_arr, osp.join(save_root, 'id', f'{mode}_logits.npy'))
    save_arr_to_dir(feature_arr, osp.join(save_root, 'id', f'{mode}_feature.npy'))
    save_arr_to_dir(label_arr, osp.join(save_root, 'id', f'{mode}_labels.npy'))

[]


Extracting reults...: 100%|██████████| 45/45 [00:02<00:00, 22.13it/s]


[]


Extracting reults...: 100%|██████████| 5/5 [00:00<00:00, 10.48it/s]


In [40]:
# save ood results
net.eval()
ood_splits = ['nearood', 'farood']
for ood_split in ood_splits:
    for dataset_name, ood_dl in ood_loader_dict[ood_split].items():
        dataiter = iter(ood_dl)
    
        logits_list = []
        feature_list = []
        label_list = []

        for i in tqdm(range(1,
                        len(dataiter) + 1),
                        desc='Extracting reults...',
                        position=0,
                        leave=True):
            batch = next(dataiter)
            data = batch['data'].cuda()
            label = batch['label']

            with torch.no_grad():
                logits_cls, feature = net(data, return_feature=True)
            logits_list.append(logits_cls.data.to('cpu').numpy())
            feature_list.append(feature.data.to('cpu').numpy())
            label_list.append(label.numpy())

        logits_arr = np.concatenate(logits_list)
        feature_arr = np.concatenate(feature_list)
        label_arr = np.concatenate(label_list)

        save_arr_to_dir(logits_arr, osp.join(save_root, ood_split, f'{dataset_name}_logits.npy'))
        save_arr_to_dir(feature_arr, osp.join(save_root, ood_split, f'{dataset_name}_feature.npy'))
        save_arr_to_dir(label_arr, osp.join(save_root, ood_split, f'{dataset_name}_labels.npy'))

Extracting reults...: 100%|██████████| 264/264 [00:08<00:00, 30.06it/s]


## MSP Evaluation

In [41]:
# # build msp method (pass in pre-saved logits)
# def msp_postprocess(logits):
#     score = torch.softmax(logits, dim=1)
#     conf, pred = torch.max(score, dim=1)
#     return pred, conf

# def msp_aug_postprocess(logits):
#     score = torch.softmax(logits, dim=1)
#     if score.shape[0] % 11 == 0: 
#         first_dim=int(score.shaape[0]/11)
#         second_dim=11
#     else:
#         print(score.shape)
#     score_reshaped=score.numpy().reshape(first_dim,second_dim,10)
#     score_averaged=torch.Tensor(np.mean(score_reshaped,axis=1))
#     conf, pred = torch.max(score_averaged, dim=1)
#     return pred, conf
from openood.postprocessors.odin_postprocessor import ODINPostprocessor

In [42]:
# load logits, feature, label for this benchmark
results = dict()
# for id
modes = ['val', 'test']
results['id'] = dict()
for mode in modes:
    results['id'][mode] = dict()
    results['id'][mode]['feature'] = np.load(osp.join(save_root, 'id', f'{mode}_feature.npy'))
    results['id'][mode]['logits'] = np.load(osp.join(save_root, 'id', f'{mode}_logits.npy'))
    results['id'][mode]['labels'] = np.load(osp.join(save_root, 'id', f'{mode}_labels.npy'))

# for ood
split_types = ['nearood', 'farood']
for split_type in split_types:
    results[split_type] = dict()
    dataset_names = config['ood_dataset'][split_type].datasets
    for dataset_name in dataset_names:
        results[split_type][dataset_name] = dict()
        results[split_type][dataset_name]['feature'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_feature.npy'))
        results[split_type][dataset_name]['logits'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_logits.npy'))
        results[split_type][dataset_name]['labels'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_labels.npy'))
        


In [43]:
# print(results['nearood']['cifar10']['logits'].shape)

In [44]:
def print_nested_dict(dict_obj, indent = 0):
    ''' Pretty Print nested dictionary with given indent level  
    '''
    # Iterate over all key-value pairs of dictionary
    for key, value in dict_obj.items():
        # If value is dict type, then print nested dict 
        if isinstance(value, dict):
            print(' ' * indent, key, ':', '{')
            print_nested_dict(value, indent + 2)
            print(' ' * indent, '}')
        else:
            print(' ' * indent, key, ':', value.shape)


In [45]:
print_nested_dict(results)

 id : {
   val : {
     feature : (1000, 512)
     logits : (1000, 100)
     labels : (1000,)
   }
   test : {
     feature : (9000, 512)
     logits : (9000, 100)
     labels : (9000,)
   }
 }
 nearood : {
   cifar10 : {
     feature : (10000, 512)
     logits : (10000, 100)
     labels : (10000,)
   }
   tin : {
     feature : (7498, 512)
     logits : (7498, 100)
     labels : (7498,)
   }
 }
 farood : {
   mnist : {
     feature : (70000, 512)
     logits : (70000, 100)
     labels : (70000,)
   }
   svhn : {
     feature : (26032, 512)
     logits : (26032, 100)
     labels : (26032,)
   }
   texture : {
     feature : (5640, 512)
     logits : (5640, 100)
     labels : (5640,)
   }
   places365 : {
     feature : (33773, 512)
     logits : (33773, 100)
     labels : (33773,)
   }
 }


In [46]:
# get pred, conf, gt from MSP postprocessor (can change to your custom_postprocessor here)
postprocessor=ODINPostprocessor(config)
postprocess_results = dict()
# id
modes = ['val', 'test']
postprocess_results['id'] = dict()
for mode in modes:
    pred, conf = postprocessor.postprocess(net=net,data=torch.from_numpy(results['id'][mode]['logits']))
    pred, conf = pred.numpy(), conf.numpy()
    gt = results['id'][mode]['labels']
    postprocess_results['id'][mode] = [pred, conf, gt]

# ood
split_types = ['nearood', 'farood']
for split_type in split_types:
    postprocess_results[split_type] = dict()
    dataset_names = config['ood_dataset'][split_type].datasets
    for dataset_name in dataset_names:
        pred, conf = postprocessor.postprocess(net=net,data=torch.from_numpy(results[split_type][dataset_name]['logits']))
        pred, conf = pred.numpy(), conf.numpy()
        gt = results[split_type][dataset_name]['labels']
        gt = -1 * np.ones_like(gt)   # hard set to -1 here
        postprocess_results[split_type][dataset_name] = [pred, conf, gt]

AttributeError: 'NoneType' object has no attribute 'temperature'

In [ ]:
def print_all_metrics(metrics):
    [fpr, auroc, aupr_in, aupr_out,
        ccr_4, ccr_3, ccr_2, ccr_1, accuracy] \
        = metrics
    print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
            end=' ',
            flush=True)
    print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
        100 * aupr_in, 100 * aupr_out),
            flush=True)
    print('CCR: {:.2f}, {:.2f}, {:.2f}, {:.2f},'.format(
        ccr_4 * 100, ccr_3 * 100, ccr_2 * 100, ccr_1 * 100),
            end=' ',
            flush=True)
    print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
    print(u'\u2500' * 70, flush=True)   

In [ ]:
from openood.evaluators.metrics import compute_all_metrics
def eval_ood(postprocess_results):
    [id_pred, id_conf, id_gt] = postprocess_results['id']['test']
    split_types = ['nearood', 'farood']

    for split_type in split_types:
        metrics_list = []
        print(f"Performing evaluation on {split_type} datasets...")
        dataset_names = config['ood_dataset'][split_type].datasets
        
        for dataset_name in dataset_names:
            [ood_pred, ood_conf, ood_gt] = postprocess_results[split_type][dataset_name]

            pred = np.concatenate([id_pred, ood_pred])
            conf = np.concatenate([id_conf, ood_conf])
            label = np.concatenate([id_gt, ood_gt])
            print(f'Computing metrics on {dataset_name} dataset...')

            ood_metrics = compute_all_metrics(conf, label, pred)
            print_all_metrics(ood_metrics)
            metrics_list.append(ood_metrics)
        print('Computing mean metrics...', flush=True)
        metrics_list = np.array(metrics_list)
        metrics_mean = np.mean(metrics_list, axis=0)   
        print_all_metrics(metrics_mean)

 

In [ ]:
print("THIS IS NON RAND AUGMENT")
eval_ood(postprocess_results)